# Bodo Getting Started Tutorial

Bodo is the simplest and most efficient analytics engine. It accelerates and scales data science programs
automatically and enables instant deployment, eliminating the need to rewrite Python analytics code to Spark/Scala, SQL or MPI/C++.
In this tutorial, we will cover the basics of using Bodo and explain how it works under the hoods.

In a nutshell, Bodo provides a just-in-time (JIT) compilation workflow using the @bodo.jit decorator. It replaces the decorated Python functions with an optimized and parallelized binary version using advanced compilation methods.

Let's get started!

## Environment Setup
Make sure MPI engines are started in the `IPython Clusters` tab (or using `ipcluster start -n 4 --profile=mpi`), then initialize the `ipyparallel` environment:

In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
view = c[:]
view.activate()

## Pi Example
Let's start with a simple example, which computes the value of Pi using Monte-Carlo Integration, to get familiar with the execution environment. Here is the Python version:

In [2]:
import numpy as np
import time

def calc_pi(n):
    t1 = time.time()
    x = 2 * np.random.ranf(n) - 1
    y = 2 * np.random.ranf(n) - 1
    pi = 4 * np.sum(x**2 + y**2 < 1) / n
    print("Execution time:", time.time()-t1, "\nresult:", pi)
    return pi

calc_pi(2 * 10**8)

Execution time: 5.6219446659088135 
result: 3.14153886


3.14153886

Now let's add the @bodo.jit decorator and run it (without parallel engines):

In [3]:
import bodo

@bodo.jit
def calc_pi(n):
    t1 = time.time()
    x = 2 * np.random.ranf(n) - 1
    y = 2 * np.random.ranf(n) - 1
    pi = 4 * np.sum(x**2 + y**2 < 1) / n
    print("Execution time:", time.time()-t1, "\nresult:", pi)
    return pi

calc_pi(2 * 10**8)

Execution time: 2.2280156369997712 
result: 3.14161316


3.14161316

We see significant speedup due to compiler optimization. Now let's use the parallel engines using the `%%px` magic:

In [4]:
%%px --block
import bodo
import numpy as np
import time

@bodo.jit
def calc_pi(n):
    t1 = time.time()
    x = 2 * np.random.ranf(n) - 1
    y = 2 * np.random.ranf(n) - 1
    pi = 4 * np.sum(x**2 + y**2 < 1) / n
    print("Execution time:", time.time()-t1, "\nresult:", pi)
    return pi

p = calc_pi(2 * 10**8)

[stdout:0] 
Execution time: 0.33458338799937337 
result: 3.14172884


Bodo automatically parallelizes this code and distributes the work among parallel engines. Hence, we see additional speedup depending on the number of cores used by engines.

## File Input
<img style="float: right;" src="img/file-read.jpg">
Efficient parallel data processing requries data I/O to be parallelized effectively as well. Bodo provides parallel file I/O for different formats such as CSV, Parquet, and HDF5 (not covered in this tutorial). This diagram demonstrates how chunks of data are partitioned among execution engines by Bodo.

### Parquet
Parquet is a commonly used file format in analytics due to its efficient columnar storage.
Let's read a Parquet file:


In [5]:
%%px --block
import pandas as pd

@bodo.jit
def pq_read():
    df = pd.read_parquet('cycling_dataset.pq')
    return df.head()

pq_read()

,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


,Unnamed: 0,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0,185.800003,51,3.46,81,30.313309,-97.732711,45,3.459,2016-10-20 22:01:26
1,1,185.800003,68,7.17,82,30.313277,-97.732715,0,3.710,2016-10-20 22:01:27
2,2,186.399994,38,11.04,82,30.313243,-97.732717,42,3.874,2016-10-20 22:01:28
3,3,186.800003,38,15.18,83,30.313212,-97.732720,5,4.135,2016-10-20 22:01:29
4,4,186.600006,38,19.43,83,30.313172,-97.732723,1,4.250,2016-10-20 22:01:30


Bodo parallelizes `pd.read_parquet` and reads equal chunks of dataframe data into the engines. Hence, the returned `df` dataframe is fully distributed and ready for scalable computation.

### CSV
Let's read the same data from a CSV file:

In [6]:
%%px --block

@bodo.jit
def csv_example(fname):
    coltypes = {'altitude': np.float64,
                'cadence': np.float64,
                'distance': np.float64,
                'hr': np.float64,
                'latitude': np.float64,
                'longitude': np.float64,
                'power': np.float64,
                'speed': np.float64,
                'time': str}
    df = pd.read_csv(fname, names=coltypes.keys(), dtype=coltypes)
    return df.head()

fname = 'cycling_dataset.csv'
csv_example(fname)

,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


,altitude,cadence,distance,hr,latitude,longitude,power,speed,time
0,0.0,0.0,185.800003,51.0,3.46,81.0,30.313309,-97.732711,45
1,1.0,1.0,185.800003,68.0,7.17,82.0,30.313277,-97.732715,0
2,2.0,2.0,186.399994,38.0,11.04,82.0,30.313243,-97.732717,42
3,3.0,3.0,186.800003,38.0,15.18,83.0,30.313212,-97.732720,5
4,4.0,4.0,186.600006,38.0,19.43,83.0,30.313172,-97.732723,1


In this example, we need to provide column types since file name is not a constant in the function to allow Bodo to know the data types during compilation.

<img style="float: right;" src="img/data-parallel.jpg">

## Data-Parallel Operations
Many operations in Numpy and Pandas are fully data-parallel, which let's Bodo parallelize them across data blocks without communication between processors.
Examples are many of math operators, filtering, combining columns, normlization, dropping rows/columns, etc.

Let's drop some rows and columns, and create a new column by extracting month from the time column.

In [7]:
%%px --block

@bodo.jit
def data_par():
    df = pd.read_parquet('cycling_dataset.pq')
    df = df[df.power!=0]
    df['month'] = df.time.dt.month
    df = df.drop(['latitude', 'longitude', 'power', 'time'], axis=1)
    return df.head()

data_par()

,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


,Unnamed: 0,altitude,cadence,distance,hr,speed,month
0,0,185.800003,51,3.460000,81,3.459,10
2,2,186.399994,38,11.040000,82,3.874,10
3,3,186.800003,38,15.180000,83,4.135,10
4,4,186.600006,38,19.430000,83,4.250,10
12,12,186.199997,0,51.610001,80,3.029,10


<img style="float: right;" src="img/reduction.jpg">

## Reduction operations
Some operators such as `sum` require reduction operation across all of data, which implies communication across data blocks. Bodo handles these operations using efficient MPI communication, and makes the output available on all processors.

As an example let's compute the mean of the 'power' column.

In [8]:
%%px --block

@bodo.jit
def mean_power():
    df = pd.read_parquet('cycling_dataset.pq')
    return df.power.mean()

print(mean_power())

[stdout:0] 102.07842132239877
[stdout:1] 102.07842132239877
[stdout:2] 102.07842132239877
[stdout:3] 102.07842132239877
[stdout:4] 102.07842132239877
[stdout:5] 102.07842132239877
[stdout:6] 102.07842132239877
[stdout:7] 102.07842132239877


<img style="float: right;" src="img/groupby.jpg">

## GroupBy/Aggregation
Grouping operations, which are typically followed by aggregations/reductions, are
more challenging for parallel and distributed environments. Bodo uses efficient MPI communication primitives to provide fast and scalable groupby/aggregations.

Let's compute the average power output per hour:

In [9]:
%%px --block
import pandas as pd
import numpy as np
import bodo

@bodo.jit
def mean_power_pm():
    df = pd.read_parquet('cycling_dataset.pq')
    df['hour'] = df.time.dt.hour
    grp = df.groupby('hour')
    mean_df = grp['power'].mean()
    return mean_df.head()

mean_power_pm()

Out[0:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[1:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[2:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[3:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[4:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[5:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[6:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

Out[7:6]: 
22    110.625821
23     71.754079
Name: power, dtype: float64

<img style="float: right;" src="img/rolling.jpg">

## Sliding Windows
Some popular analytics operations, especially for time-series analysis, are based on sliding windows. Examples include moving averages and percentage change. In a distributed setup, these require communication beyond map-reduce (which is the basis of most systems such as Spark). Bodo handles these cases using efficient patterns known from HPC.

Let's compute the moving average of the heart-rate.

In [10]:
%%px --block

@bodo.jit
def mov_avg():
    df = pd.read_parquet('cycling_dataset.pq')
    mv_av = df.hr.rolling(4).mean()
    return mv_av.head()

mov_avg()

Out[0:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[1:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[2:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[3:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[4:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[5:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[6:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

Out[7:7]: 
0     NaN
1     NaN
2     NaN
3    82.0
4    82.5
Name: hr, dtype: float64

## Join
Bodo can also efficiently join dataframes, which uses a communication pattern similar to Groupby.

Let's read data, split into 2 dataframes and re-join on time column.

In [11]:
%%px --block

@bodo.jit
def merge_dfs():
    df = pd.read_parquet('cycling_dataset.pq')
    df1 = df[['altitude', 'cadence', 'distance', 'hr', 'time']]
    df2 = df[['latitude', 'longitude', 'power', 'speed', 'time']]
    df3 = df1.merge(df2, on='time')
    return df3.head()

merge_dfs()

,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


,altitude,cadence,distance,hr,time,latitude,longitude,power,speed
0,185.800003,51,3.46,81,2016-10-20 22:01:26,30.313309,-97.732711,45,3.459
1,185.800003,68,7.17,82,2016-10-20 22:01:27,30.313277,-97.732715,0,3.710
2,186.399994,38,11.04,82,2016-10-20 22:01:28,30.313243,-97.732717,42,3.874
3,186.800003,38,15.18,83,2016-10-20 22:01:29,30.313212,-97.732720,5,4.135
4,186.600006,38,19.43,83,2016-10-20 22:01:30,30.313172,-97.732723,1,4.250


## Automatic Distribution

Bodo automatically distributes data and computation of the target function by analyzing it for parallelization. It chooses the best and *safest* possible distribution. For example, returning distributed data is not necessary *safe*, since the code outside of the bodo scope would need to handle chunks of data instead of the full data. Consider the example below:

In [12]:
%%px --block

@bodo.jit
def read_pq():
    df = pd.read_parquet('cycling_dataset.pq')
    return df

df = read_pq()
print(df.shape)
read_pq.distributed_diagnostics()

[stdout:0] 
(3902, 10)
Distributed diagnostics for function read_pq, <ipython-input-9-c06d4064301d> (1)

Data distributions:
   Unnamed: 0.10382           REP
   altitude.10383             REP
   cadence.10384              REP
   distance.10385             REP
   hr.10386                   REP
   latitude.10387             REP
   longitude.10388            REP
   power.10389                REP
   speed.10390                REP
   time.10391                 REP
   __index_level_0__.10392    REP
   $0.15.10408                [<Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>, <Distribution.REP: 1>]
   $0.23.10514                REP
   $df.10547                  REP
   $0.6                       REP

Parfor distributions:
No parfors to distribute.

Distributed listing for function read_pq, <ipython-input-9-c06d4064301d> (1)
-----------

The `distributed_diagnostics` function prints diagnostics information about distribution analysis by Bodo. In this case, all variables are assigned the `REP` distribution, which means they are replicated and there is no distribution of data. The reason is the return of `df`, which also propagates `REP` to all other variables since they are involved in parallel computation with `df`.

We can change this behavior by a simple annotation for `df`:

In [13]:
%%px --block

@bodo.jit(distributed=['df'])
def read_pq():
    df = pd.read_parquet('cycling_dataset.pq')
    return df

df = read_pq()
print(df.shape)
read_pq.distributed_diagnostics()

[stdout:0] 
(488, 10)
Distributed diagnostics for function read_pq, <ipython-input-10-dda0820ca7f5> (1)

Data distributions:
   Unnamed: 0.10741            1D_Block
   altitude.10742              1D_Block
   cadence.10743               1D_Block
   distance.10744              1D_Block
   hr.10745                    1D_Block
   latitude.10746              1D_Block
   longitude.10747             1D_Block
   power.10748                 1D_Block
   speed.10749                 1D_Block
   time.10750                  1D_Block
   __index_level_0__.10751     1D_Block
   $0.15.10767                 [<Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>, <Distribution.OneD: 5>]
   $0.23.10883                 1D_Block
   $df.10916                   1D_Block
   distributed_return.10810    1D_Block
   $dist_return.10807.10917    1D_Block

Pa

In this case, all variables are assigned the `1D_Block` distribution, which means they are divided in equal chunks among processors. The returned dataframe on each processor is therefore a chunk of the full dataset. This is useful, for example, when computation on chunks is desired outside the scope of Bodo (e.g. mixing Bodo code with custom non-Bodo code and other packages like TensorFlow).